In [2]:
import torch

In [ ]:
# Define the base linear layer


class Linear:

    def __init__(
        self,
        fan_in: float,
        fan_out: float,
        bias: bool = True,
        generator: "object" = None,
    ):
        kaiming_init = fan_in**0.5
        self.weight = torch.randn((fan_in, fan_out), generator=generator) / kaiming_init
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x: "torch.tensor") -> "torch.tensor":
        out = x @ self.weight
        if self.bias is not None:
            out += self.bias
        return out

    @property
    def parameters(self) -> "list[torch.tensor]":
        return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1D:

    def __init__(self, dim: int, eps: float = 1e-5, momentum: float = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True

        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    # TODO: Define __call__

    # TODO: Define parameters property